In [1]:
import symmetric_svd_word_embedding_utilities as util

# Parameters

In [2]:
# whether consider the order of pmi
pmi_order = False

# threshold of rare unigram
rare_term = 10

# threshold of rare bigram
rare_bigram = 5

# dimension of matrix generated
dimension = 100

# value applied in laplace smooth
la_smooth = 1

# number of tweets deal with
num_of_tweets = 1000

# Import data

In [3]:
mypath = './train_tweets'
header_row = ['text','label']

In [4]:
df_raw = util.read_from_txt(mypath)

the filenames are ***************************
['tweets_senEval.txt']
Original size of data is  49828
after remove duplicate, the size is  49826




In [5]:
# mypath = '../tweets_set_ire'
# header_row = ['ID', 'ID_STR', 'author', 'text', 'time', 'location', 'latitude', 'longitude', 'screen_place']

In [6]:
# df_raw = util.read_from_csv(mypath,header_row)

In [7]:
print('the first row is*******************************')
print(df_raw.iloc[0])
print("**********************************************")

the first row is*******************************
text     Won the match #getin . Plus tomorrow is a very...
label                                              neutral
Name: 0, dtype: object
**********************************************


# Preprocessing tweet, remove mention, urls, emoji, strip each token and so on

In [8]:
# tweet_raw = util.preprocess_tweet(df_raw['text'][:num_of_tweets])
tweet_raw_ = util.preprocess_tweet(df_raw['text'])

In [9]:
tweet_raw=[]
for idx,row in df_raw.iterrows():
    if row['label'] == 'neutral':
        tweet_raw.append(tweet_raw_[idx]+['neutral_corpus'])
    elif row['label'] == 'positive':
        tweet_raw.append(tweet_raw_[idx]+['positive_corpus'])
    else:
        tweet_raw.append(tweet_raw_[idx]+['negative_corpus'])

In [10]:
print('the number of tweets are ****************************************',len(tweet_raw))
print("**********************************************")

the number of tweets are **************************************** 49826
**********************************************


In [11]:
print('first 5 tweets after preprocessing********************************')
print(tweet_raw[:5])
print("**********************************************")

first 5 tweets after preprocessing********************************
[['won', 'the', 'match', 'getin', '', 'plus', 'tomorrow', 'is', 'a', 'very', 'busy', 'day', 'with', 'awareness', "day's", 'and', 'debates', 'gulp', 'debates', 'neutral_corpus'], ['some', 'areas', 'of', 'new', 'england', 'could', 'see', 'the', 'first', 'flakes', 'of', 'the', 'season', 'tuesday', 'neutral_corpus'], ['2nd', 'worst', 'qb', 'definitely', 'tony', 'romo', 'the', 'man', 'who', 'likes', 'to', 'share', 'the', 'ball', 'with', 'everyone', 'including', 'the', 'other', 'team', 'negative_corpus'], ['thailand', 'washington', '', 'us', 'president', 'barack', 'obama', 'vowed', 'wednesday', 'as', 'he', 'visited', 'storm-ravaged', 'new', 'jersey', 'shore', 'to', 'neutral_corpus'], ['did', "y'all", 'hear', 'what', 'tony', 'romo', 'dressed', 'up', 'as', 'for', 'halloween', 'a', 'giants', 'quaterback', 'cause', "that's", 'all', 'he', 'could', 'throw', 'to', 'sunday', 'night', 'neutral_corpus']]
*******************************

# Compute the frequency of each term, remove the rare terms and stop words

In [12]:
punctuation = list(util.string.punctuation)
stop = util.stopwords.words('english') + punctuation + ['the','rt','&amp','a','','amp']
print('stop words are****************************************************')
print(stop)
print("**********************************************")

stop words are****************************************************
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some',

In [13]:
## count all terms frequency before removing stop words etc.
count_before_remove = util.word_counter(tweet_raw)

In [14]:
print('top 20 frequency words before removing*****************************')
print(count_before_remove.most_common(20))
print("**********************************************")
print("size of vocabulary before removing is ******************************",len(count_before_remove))
print("**********************************************")

top 20 frequency words before removing*****************************
[('the', 41256), ('to', 22876), ('neutral_corpus', 22333), ('positive_corpus', 19726), ('', 18639), ('in', 14827), ('on', 14264), ('a', 14081), ('i', 13869), ('and', 13777), ('of', 11858), ('for', 10831), ('is', 10606), ('you', 8904), ('with', 8258), ('be', 7994), ('tomorrow', 7792), ('negative_corpus', 7767), ('at', 7647), ('it', 7392)]
**********************************************
size of vocabulary before removing is ****************************** 53355
**********************************************


In [15]:
tweet_final = util.remove_rare_stop_words(tweet_raw,count_before_remove,rare_term,stop)

In [16]:
print('first 5 tweets after removing rare term and stop words**************')
print(tweet_final[:5])
print("**********************************************")

first 5 tweets after removing rare term and stop words**************
[['match', 'plus', 'tomorrow', 'busy', 'day', 'awareness', "day's", 'neutral_corpus'], ['areas', 'new', 'england', 'could', 'see', 'first', 'season', 'tuesday', 'neutral_corpus'], ['2nd', 'worst', 'qb', 'definitely', 'tony', 'romo', 'man', 'likes', 'share', 'ball', 'everyone', 'including', 'team', 'negative_corpus'], ['washington', 'us', 'president', 'barack', 'obama', 'wednesday', 'visited', 'new', 'jersey', 'shore', 'neutral_corpus'], ["y'all", 'hear', 'tony', 'romo', 'dressed', 'halloween', 'giants', 'cause', "that's", 'could', 'throw', 'sunday', 'night', 'neutral_corpus']]
**********************************************


In [17]:
# count all terms frequency after removing stop words etc.
counter_uni = util.word_counter(tweet_final)

In [18]:
print('top 20 frequency words after removing*******************************')
print(counter_uni.most_common(20))
print("**********************************************")
print("size of vocabulary after removing is",len(counter_uni))

top 20 frequency words after removing*******************************
[('neutral_corpus', 22333), ('positive_corpus', 19726), ('tomorrow', 7792), ('negative_corpus', 7767), ('may', 7354), ('day', 4182), ('going', 3293), ('night', 3134), ('see', 3081), ("i'm", 3035), ('friday', 2949), ('1st', 2845), ('sunday', 2761), ('like', 2722), ('time', 2576), ('get', 2464), ('saturday', 2218), ('go', 2125), ('one', 2123), ('new', 2035)]
**********************************************
size of vocabulary after removing is 6012


# Count the co-occurrence of any two tokens

In [19]:
count_collocation = util.collocation_counter(tweet_final,pmi_order)

In [20]:
print('the top 20 freqency collocation bigrams are **********************')
print(count_collocation.most_common(20))
print("**********************************************")
print('size of bi-gram counter is ',len(count_collocation))
print("**********************************************")

the top 20 freqency collocation bigrams are **********************
[(('tomorrow', 'positive_corpus'), 3740), (('may', 'neutral_corpus'), 3180), (('tomorrow', 'neutral_corpus'), 3058), (('may', 'positive_corpus'), 2474), (('day', 'positive_corpus'), 2312), (('see', 'positive_corpus'), 1724), (('may', 'negative_corpus'), 1700), (('night', 'positive_corpus'), 1519), (("i'm", 'positive_corpus'), 1450), (('going', 'neutral_corpus'), 1426), (('going', 'positive_corpus'), 1400), (('day', 'neutral_corpus'), 1388), (('friday', 'positive_corpus'), 1383), (('night', 'neutral_corpus'), 1320), (('good', 'positive_corpus'), 1267), (('friday', 'neutral_corpus'), 1260), (('sunday', 'neutral_corpus'), 1252), (('1st', 'neutral_corpus'), 1244), (('going', 'tomorrow'), 1233), (('sunday', 'positive_corpus'), 1170)]
**********************************************
size of bi-gram counter is  1057268
**********************************************


In [21]:
## remove the low frequency bigrams
counter_col = util.remove_rare_term(count_collocation,rare_bigram)

In [22]:
print('size of bi-gram counter after removing rare bigrame is ',len(counter_col))
print("**********************************************")

size of bi-gram counter after removing rare bigrame is  70477
**********************************************


In [23]:
sorted_x = sorted(counter_col.items(), key=util.operator.itemgetter(1))
sorted_x.reverse()

In [24]:
print('the top 10 colloation words are*********************************')
print(sorted_x[:10])
print("**********************************************")

sorted_x.reverse()
print('the most low 10 colloation r words are*********************************')
print(sorted_x[:10])
print("**********************************************")

the top 10 colloation words are*********************************
[(('tomorrow', 'positive_corpus'), 3740), (('may', 'neutral_corpus'), 3180), (('tomorrow', 'neutral_corpus'), 3058), (('may', 'positive_corpus'), 2474), (('day', 'positive_corpus'), 2312), (('see', 'positive_corpus'), 1724), (('may', 'negative_corpus'), 1700), (('night', 'positive_corpus'), 1519), (("i'm", 'positive_corpus'), 1450), (('going', 'neutral_corpus'), 1426)]
**********************************************
the most low 10 colloation r words are*********************************
[(('2nd', 'share'), 6), (('2nd', 'including'), 6), (('worst', 'team'), 6), (('including', 'negative_corpus'), 6), (('hear', "that's"), 6), (('hear', 'could'), 6), (('tony', 'night'), 6), (('romo', 'sunday'), 6), (('cause', 'sunday'), 6), (('tina', 'january'), 6)]
**********************************************


# Construct word occurrences matrix and PMI matrix

In [25]:
# document = [['A', 'B'], ['C', 'B'], ['A', 'B', 'C', 'D']]
# counter_uni = util.word_counter(document)
# counter_col = util.collocation_counter(document)
df_coll,matrix_coll = util.construct_word_occurrence_matrix(counter_uni,counter_col)

In [26]:
print(df_coll)

                 match  plus  tomorrow  busy     day  awareness  day's  \
match             21.0   0.0      39.0   0.0     7.0        0.0    0.0   
plus               0.0   0.0      22.0   0.0     9.0        0.0    0.0   
tomorrow          39.0  22.0     104.0  35.0  1004.0        7.0    0.0   
busy               0.0   0.0      35.0   0.0    29.0        0.0    0.0   
day                7.0   9.0    1004.0  29.0   517.0       15.0    0.0   
awareness          0.0   0.0       7.0   0.0    15.0        0.0    0.0   
day's              0.0   0.0       0.0   0.0     0.0        0.0    0.0   
neutral_corpus   166.0  39.0    3058.0  42.0  1388.0       11.0    0.0   
areas              0.0   0.0       0.0   0.0     0.0        0.0    0.0   
new                0.0  11.0     315.0   0.0   144.0        0.0    0.0   
england            0.0   0.0       7.0   0.0     0.0        0.0    0.0   
could              9.0   0.0      90.0   0.0    45.0        0.0    0.0   
see               13.0  10.0     847.0

In [27]:
names_index = df_coll.index.tolist()
names_header = df_coll.index.tolist()

In [28]:
dff_pmi,ppmi_matrix = util.PPMI_matrix(names_index,names_header,matrix_coll,la_smooth)

applying laplace smoothed, parameter is 1


In [29]:
print(dff_pmi)

                    match      plus  tomorrow      busy       day  awareness  \
match            3.806133  0.000000  1.484869  0.000000  0.000000   0.000000   
plus             0.000000  0.000000  0.979114  0.000000  0.508005   0.006007   
tomorrow         1.484869  0.979114  0.000000  1.657060  3.682685   0.000000   
busy             0.000000  0.000000  1.657060  0.000000  2.124551   0.037591   
day              0.000000  0.508005  3.682685  2.124551  3.457040   1.260161   
awareness        0.000000  0.006007  0.000000  0.037591  1.260161   0.080091   
day's            0.000000  0.012695  0.000000  0.044279  0.000000   0.086779   
neutral_corpus   1.946663  0.177497  2.958042  0.313417  2.549554   0.000000   
areas            0.000000  0.013653  0.000000  0.045237  0.000000   0.087737   
new              0.000000  1.714682  2.957129  0.000000  2.563783   0.000000   
england          0.000000  0.000000  0.000000  0.000000  0.000000   0.023774   
could            2.501459  0.000000  2.5

In [30]:
print('the highest PPMI value tuples is****************************')
print(dff_pmi.stack().index[util.np.argmax(ppmi_matrix)])
print()

the highest PPMI value tuples is****************************
('hogan', 'hulk')



# Apply symmetrics SVD to get the word and context embedding matrix

In [31]:
W,C = util.construct_word_embedding(dimension,ppmi_matrix)

In [32]:
print(W)

[[-6.16366987e-01  2.24955101e-01 -2.92381746e-01 ...  4.61465890e-02
  -2.97033399e-01 -1.52938188e-01]
 [-1.32775129e-01  1.62248474e-02 -2.09789931e-01 ...  1.12382337e-01
  -5.24645547e-02  6.80586015e-02]
 [-1.84025695e+00  6.29359755e-01  1.55949558e+00 ...  1.80931210e-01
   1.05105315e-01  1.00406231e-01]
 ...
 [-1.04851923e-01 -2.95116581e-01  7.05089457e-03 ...  1.38731712e-03
  -9.81751919e-05  1.07269029e-03]
 [-1.09314476e-01 -9.31641886e-03 -1.21067767e-01 ... -3.17927192e-02
  -1.15450535e-01  7.64422321e-02]
 [-8.95848471e-02 -2.51409662e-01  7.49018938e-03 ...  5.62255363e-03
  -1.89250776e-02 -1.26025147e-02]]


In [33]:
print('shape of W is**********************************************')
print(W.shape)
print("**********************************************")

print('shape of C is*********************************************')
print(C.shape)
print("**********************************************")

shape of W is**********************************************
(6012, 100)
**********************************************
shape of C is*********************************************
(6012, 100)
**********************************************


In [34]:
header = dff_pmi.index.tolist()
frame=util.to_csv(W,header,'test_svd_embedding.csv')
print('export word embedding to csv file!!!')

export word embedding to csv file!!!


In [35]:
from scipy import spatial

dataSetI = [3, 45, 7, 2]
dataSetII = [3, 54,7, 2]
result = spatial.distance.cosine(dataSetI, dataSetII)

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
cos_sim_matrix = cosine_similarity(frame)

In [38]:
frame1=frame.reset_index(drop=False)

In [64]:
frame1[frame1['index'] == 'neutral_corpus'].index

Int64Index([7], dtype='int64')

In [65]:
cos_sim_matrix[64]

array([-0.01797303,  0.10714496, -0.06409525, ...,  0.87612823,
        0.08294716,  0.85315428])

In [76]:
import numpy as np
arr = np.array(cos_sim_matrix[56])

order = arr.argsort()[-100:][::-1]
order

array([  56,   12,    4,  157,   90,  166,   76,   58,  457,   83,    2,
          7,   48,   59,  156,  399,  179,  103,  143,    9,  219,  210,
        309,  349,  155,  479,   77,   47,   13,  414,  290,  154,  483,
         70,   16,  209,  129,  421,  107,  214,   31,  342,  291,  415,
        120,  108,  313,  731,   51,  435,  408,  468,   29,  594,  838,
        416,  598,  254,  486,  260,  105,  489,  304,   81, 1006,  109,
        279,  318,  272, 1070,  581,  625,  434,  218,  308,  123,  665,
       1119,   26,  378, 1246,  199,  446,  351,  306,  208,  813,  782,
        148,  255, 2336,  354,  343, 1134, 1188,  139,   22,  161,  165,
        733])

In [77]:
for i in order:
    print(frame1.loc[i]['index'])

positive_corpus
see
day
one
get
time
going
good
today
i'm
tomorrow
neutral_corpus
night
love
like
go
friday
saturday
last
new
tonight
come
great
best
happy
know
back
sunday
first
1st
can't
want
really
think
2nd
still
got
make
hope
2
us
birthday
wait
would
monday
next
coming
3
may
watch
gonna
show
negative_corpus
big
much
better
live
3rd
i'll
well
game
morning
weekend
work
fun
thursday
play
watching
1
year
way
right
free
excited
concert
need
ever
also
everyone
even
amazing
4
lol
getting
sat
music
seeing
every
since
saw
enjoy
win
september
song
guys
week
man
playing
4th
hey


In [73]:
frame1.shape

(6012, 101)